In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2
import os
import pathlib
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

import pymysql

from sklearn.model_selection import train_test_split
import pickle

In [2]:
data_path = "..\\student_images\\"

## Saving image of new student in new folder and also appending its path in dictionary

In [11]:
def create_student_images_dict(data_path):
    student_images_dict = {}
    # Iterate over directories in data_path
    for student_folder in os.listdir(data_path):
        student_name = student_folder
        # Get the path to the student's images
        student_image_path = os.path.join(data_path, student_folder)
        # Check if it's a directory
        if os.path.isdir(student_image_path):
            # List image files in the directory
            image_files = list(pathlib.Path(student_image_path).glob('*'))
            # Add student name and image paths to the dictionary
            student_images_dict[student_name] = image_files
    return student_images_dict


student_images_dict = create_student_images_dict(data_path)
student_images_dict

{'Ananya Rathour': [WindowsPath('../student_images/Ananya Rathour/IMG_20240317_143108.jpg'),
  WindowsPath('../student_images/Ananya Rathour/IMG_20240317_143111.jpg'),
  WindowsPath('../student_images/Ananya Rathour/IMG_20240317_143114.jpg'),
  WindowsPath('../student_images/Ananya Rathour/IMG_20240317_143119.jpg'),
  WindowsPath('../student_images/Ananya Rathour/IMG_20240317_143122.jpg'),
  WindowsPath('../student_images/Ananya Rathour/IMG_20240317_143124.jpg'),
  WindowsPath('../student_images/Ananya Rathour/IMG_20240317_143129.jpg'),
  WindowsPath('../student_images/Ananya Rathour/IMG_20240317_143132.jpg'),
  WindowsPath('../student_images/Ananya Rathour/IMG_20240317_143138.jpg')],
 'Manshi Rathour': [WindowsPath('../student_images/Manshi Rathour/IMG_20240317_143155.jpg'),
  WindowsPath('../student_images/Manshi Rathour/IMG_20240317_143204.jpg'),
  WindowsPath('../student_images/Manshi Rathour/IMG_20240317_143206.jpg'),
  WindowsPath('../student_images/Manshi Rathour/IMG_20240317_14

In [12]:
def save_images_for_new_student(new_student_name, image_data, data_path, student_images_dict):
    # Create a folder for the new student if it doesn't exist
    student_folder_path = os.path.join(data_path, new_student_name)
    os.makedirs(student_folder_path, exist_ok=True)
    
    # Save the images sent from the backend
    image_paths = []
    for i, image in enumerate(image_data):
        image_path = os.path.join(student_folder_path, f"image_{i+1}.jpg")
        with open(image_path, "wb") as file:
            file.write(image)
        image_paths.append(pathlib.Path(image_path))
    
    # Update student_images_dict with the new student folder path
    student_images_dict[new_student_name] = image_paths
    
    return student_images_dict

In [13]:
# new_student_name = "New Student"
# image_data = [b'image_1_data', b'image_2_data', b'image_3_data', b'image_4_data', b'image_5_data', b'image_6_data']
# student_images_dict = save_images_for_new_student(new_student_name, image_data, data_path, student_images_dict)
# student_images_dict 

{'Ananya Rathour': [WindowsPath('../student_images/Ananya Rathour/IMG_20240317_143108.jpg'),
  WindowsPath('../student_images/Ananya Rathour/IMG_20240317_143111.jpg'),
  WindowsPath('../student_images/Ananya Rathour/IMG_20240317_143114.jpg'),
  WindowsPath('../student_images/Ananya Rathour/IMG_20240317_143119.jpg'),
  WindowsPath('../student_images/Ananya Rathour/IMG_20240317_143122.jpg'),
  WindowsPath('../student_images/Ananya Rathour/IMG_20240317_143124.jpg'),
  WindowsPath('../student_images/Ananya Rathour/IMG_20240317_143129.jpg'),
  WindowsPath('../student_images/Ananya Rathour/IMG_20240317_143132.jpg'),
  WindowsPath('../student_images/Ananya Rathour/IMG_20240317_143138.jpg')],
 'Manshi Rathour': [WindowsPath('../student_images/Manshi Rathour/IMG_20240317_143155.jpg'),
  WindowsPath('../student_images/Manshi Rathour/IMG_20240317_143204.jpg'),
  WindowsPath('../student_images/Manshi Rathour/IMG_20240317_143206.jpg'),
  WindowsPath('../student_images/Manshi Rathour/IMG_20240317_14

## Connecting to MySQL server to fetch student name with their details

In [14]:
def load_data_from_mysql(host, user, password, database, student_images_dict):
    try:
        # Connect to MySQL without specifying the authentication plugin
        connection = pymysql.connect(
            host=host,
            user=user,
            password=password,
            database=database
        )

        # Create a cursor object
        cursor = connection.cursor()

        # Execute a query to fetch data
        query = "SELECT * FROM students"
        cursor.execute(query)

        # Fetch data and convert it into a DataFrame
        data = cursor.fetchall()
        df = pd.DataFrame(data, columns=[col[0] for col in cursor.description])

        # Convert DataFrame to dictionary
        student_dict = df.set_index('name')['student_id'].to_dict()

        # Close the cursor and connection
        cursor.close()
        connection.close()

        return student_dict

    except Exception as e:
        print(f"An error occurred: {e}")
        return None, None

    


student_dict = load_data_from_mysql(host, user, password, database, student_images_dict)


print("Student Dictionary:")
print(student_dict)


Student Dictionary:
{'Manshi Rathour': 1, 'Ananya Rathour': 2, 'Naincy Rathour': 3, 'Mayank Rathour': 4}


## Resizing images 

In [15]:
def resize_images(student_images_dict, student_dict, X=None, y=None, image_size=(180, 180)):
    if X is None:
        X = []
    if y is None:
        y = []

    processed_files = set()
    
    # Add existing files to processed_files
    for img in X:
        processed_files.add(img)
    
    for student_name, images in student_images_dict.items():
        for image in images:
            # Check if the image file exists
            if os.path.exists(str(image)):
                # Check if the image has been processed already
                if str(image) not in processed_files:
                    # Read the image
                    img = cv2.imread(str(image))
                    
                    # Check if the image was read successfully
                    if img is not None:
                        # Resize the image
                        resized_img = cv2.resize(img, image_size)
                        
                        # Convert BGR to RGB
                        resized_img_rgb = cv2.cvtColor(resized_img, cv2.COLOR_BGR2RGB)
                        
                        # Append the resized image to X
                        X.append(resized_img_rgb)
                        
                        # Append the corresponding label to y
                        y.append(student_dict[student_name])
                        
                        # Add the filename to processed_files
                        processed_files.add(str(image))
                    else:
                        print(f"Unable to read image: {image}")
                else:
                    print(f"Image already processed: {image}")
            else:
                print(f"Image file does not exist: {image}")

    # Convert lists to numpy arrays
    X = np.array(X)
    y = np.array(y)
    
    return X, y


X, y = resize_images(student_images_dict, student_dict)


print("X shape:", X.shape)
print("y shape:", y.shape)

Unable to read image: ..\student_images\New Student\image_1.jpg
Unable to read image: ..\student_images\New Student\image_2.jpg
Unable to read image: ..\student_images\New Student\image_3.jpg
Unable to read image: ..\student_images\New Student\image_4.jpg
Unable to read image: ..\student_images\New Student\image_5.jpg
Unable to read image: ..\student_images\New Student\image_6.jpg
X shape: (38, 180, 180, 3)
y shape: (38,)


## Train Test Split

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

## Processing : scale images 

In [17]:
X_train_scaled = X_train / 255
X_test_scaled = X_test / 255

# Build convolutional neural network and train it

In [18]:
data_augmentation = keras.Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal", 
                                                 input_shape=(180, 
                                                              180,
                                                              3)),
    layers.experimental.preprocessing.RandomZoom(0.1),
  ]
)

In [19]:
num_classes = 5

model = Sequential([
  data_augmentation,
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
              
model.fit(X_train_scaled, y_train, epochs=30)       



Epoch 1/30


1/1 [==============================] - 2s 2s/step - loss: 1.6472 - accuracy: 0.2857
Epoch 2/30
1/1 [==============================] - 0s 191ms/step - loss: 2.9456 - accuracy: 0.2857
Epoch 3/30
1/1 [==============================] - 0s 203ms/step - loss: 4.5900 - accuracy: 0.2857
Epoch 4/30
1/1 [==============================] - 0s 193ms/step - loss: 4.4509 - accuracy: 0.2857
Epoch 5/30
1/1 [==============================] - 0s 189ms/step - loss: 2.9532 - accuracy: 0.4286
Epoch 6/30
1/1 [==============================] - 0s 197ms/step - loss: 2.0146 - accuracy: 0.1429
Epoch 7/30
1/1 [==============================] - 0s 195ms/step - loss: 1.3275 - accuracy: 0.5357
Epoch 8/30
1/1 [==============================] - 0s 186ms/step - loss: 1.1184 - accuracy: 0.5714
Epoch 9/30
1/1 [==============================] - 0s 200ms/step - loss: 1.1682 - accuracy: 0.5357
Epoch 10/30
1/1 [==============================] - 0s 192ms/step - loss: 1.1526 - accuracy: 0.5714
Epoch 11/30
1/1 [=

In [20]:
model.evaluate(X_test_scaled,y_test)

1/1 [==============================] - 0s 203ms/step - loss: 0.1217 - accuracy: 1.0000


[0.12169454246759415, 1.0]

In [21]:
predictions = model.predict(X_test_scaled)
predictions

1/1 [==============================] - 0s 125ms/step


array([[-1.3425345e+01,  5.3855433e+00, -1.8600888e+00,  3.8618374e+00,
         8.5728703e+00],
       [-1.3016029e+01,  2.6115177e+00, -6.0269034e-01,  7.4534979e+00,
         1.2354205e+00],
       [-1.4541209e+01,  9.2354527e+00,  2.4367200e-01,  5.3720551e+00,
         3.5282743e+00],
       [-1.3114316e+01,  9.0842390e+00,  1.8418292e-02,  4.5803981e+00,
         3.4810174e+00],
       [-1.3401680e+01,  9.2818575e+00, -1.2933512e-01,  4.2467365e+00,
         4.3949089e+00],
       [-1.5086336e+01,  8.7821035e+00,  1.4756517e+00,  6.0134931e+00,
         2.5777490e+00],
       [-1.1581469e+01,  5.6319790e+00, -3.8998673e+00,  4.0307455e+00,
         6.1696796e+00],
       [-1.3775360e+01,  8.2477160e+00,  5.4447794e-01,  5.0360637e+00,
         3.7911148e+00],
       [-1.3189781e+01,  6.1017499e+00, -1.6272542e+00,  4.2550359e+00,
         6.7901697e+00],
       [-2.5451221e+01, -9.5933304e+00,  2.7255560e+01,  1.6570337e+01,
        -8.7223387e+00]], dtype=float32)

In [22]:
score = tf.nn.softmax(predictions[0])

In [23]:
np.argmax(score)

4

In [24]:
y_test[0]

4

## Save trained model to pickel file

In [25]:
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)